In [ ]:
class valuation_class(object):
    '''
    Basic class for single factor derivative valuation
    Attributes
    =========
    name: str - name of the object
    underlying: instance of a simulation class modeling single risk factor
    mar_env: instance of market environment
    payoff_func: str - derivative payoff as a python function that can be evaluated
    
    Methods
    =========
    update: updates selected valuation parameters
    delta: returns delta of derivative (change in price to underlying risk factor)
    vega: returns vega of derivative (change in price to implied volatility)
    '''
    def __init__(self, name, underlying, mar_env, payoff_func=''):
        self.name = name
        self.pricing_date = mar_env.pricing_date
        try:
            # strike price is optional
            self.strike = mar_env.get_constant('strike')
        except:
            pass
        self.maturity = mar_env.get_constant('maturity')
        self.currency = mar_env.get_constant('currency')
        # simulation parameters and discount curve from simulation object
        self.frequency = underlying.frequency
        self.paths = underlying.paths
        self.discount_curve = underlying.discount_curve
        self.payoff_func = payoff_func
        # provide pricing and maturity date to underlying
        self.underlying.special_dates.extend([self.pricing_date, self.maturity])
        
    def update(self, initial_value=None, volatility=None, strike=None, maturity=None):
        if initial_value is not None:
            self.underlying.update(initial_value=initial_value)
        if volatility is not None:
            self.underlying.update(volatility=volatility)
        if strike is not None:
            self.strike = strike
        if maturity is not None:
            self.maturity = maturity
            # add new maturity date if not already in time_grid
            if maturity not in self.underlying.time_grid:
                self.underlying.special_dates.append(maturity)
                self.underlying.instrument_values = None # because this means it is first update
                
    def delta(self, interval=None, accuracy=4):
        if interval is None:
            interval = self.underlying.initial_value / 50. # ?
            
        